## Map Mutations to Protein-Protein and Protein-Nucleic Acid Interfaces
Here we find and visualize the mutations that occur around protein-protein and protein-nucleic acid interfaces.

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionExtractor, InteractionFilter
from ipywidgets import interact, IntSlider
import py3Dmol
import pandas as pd

In [2]:
# Initialize Spark
spark = SparkSession.builder.master("local[4]").appName("3-MapToPolymerInteractions").getOrCreate()

#### Input parameters

In [3]:
distance_cutoff = 8 # distance cutoff for finding and visualizing interactions
input_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures

## Read mutation file created in the previous step

In [4]:
df = pd.read_csv(input_file_name)
df['pdbPosition'] = df['pdbPosition'].astype('str') # PDB residue numbers must be string to handle insertion codes
df.head()

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId,tax_id,scientific_name,structureChainId
0,1GUH,A,83,G,hgvs-grch37,chr6:g.52619766C>T,9606,Homo sapiens,1GUH.A
1,1XZ7,B,108,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,1XZ7.B
2,2VCT,D,83,G,hgvs-grch37,chr6:g.52619766C>T,9606,Homo sapiens,2VCT.D
3,3KMF,G,708,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,3KMF.G
4,1GSF,D,83,G,hgvs-grch37,chr6:g.52619766C>T,9606,Homo sapiens,1GSF.D


## Create a list of unique PDB Ids

In [5]:
pdb_ids = list(df.structureId.drop_duplicates())
print("PDB Ids:", pdb_ids)

PDB Ids: ['1GUH', '1XZ7', '2VCT', '3KMF', '1GSF', '1YDZ', '6ATQ', '1Y09', '6ATO', '2FO0', '1XYE', '4ACS', '2WJU', '4WA9', '2G1T', '1XY0', '4TWP', '1K3Y', '1YHE', '1OPL', '2V7A', '1YHR', '1YZI', '1QXD', '1XZV', '1VWT', '3UE4', '2R6K', '1PL1', '3QRI', '1DXT', '6FQF', '2HIW', '2GQG', '1XZ4', '6ATR', '2F4J', '1Y0A', '1Y0D', '1XZU', '2W6V', '1YH9', '2G2H', '5JCU', '4ZOG', '1XZ5', '1GSE', '1Y0C', '1Y8W', '2HZI', '1PKW', '3KTL', '3QRK', '1XXT', '1XZ2', '6ATP', '1K3L', '4XS0', '2G2F', '5MO4']


## Find all polymer-polymer interactions
Find groups (residues) that interact with other polymer chains

In [6]:
structures = mmtfReader.download_mmtf_files(pdb_ids)

queryGroupId, queryChainId, and queryGroupNumber specify the residue that interacts with another chain (targetChainId)

In [7]:
interaction_filter = InteractionFilter(distanceCutoff=distance_cutoff)

interactions = InteractionExtractor().get_polymer_interactions(structures, interaction_filter, level='chain').toPandas()
interactions.head()

,structureChainId,queryGroupId,queryChainId,queryGroupNumber,targetChainId
0,1XY0.B,LEU,A,125,B
1,1XY0.A,PRO,B,36,A
2,1XY0.B,LEU,A,100,B
3,1XY0.C,PRO,B,36,C
4,1XY0.D,PHE,C,117,D


## Filter mutations by joining with the interaction data

In [8]:
mt = df.merge(interactions, left_on=['structureChainId','pdbPosition'], right_on=['structureChainId','queryGroupNumber'], how='inner')
mt.head()

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId,tax_id,scientific_name,structureChainId,queryGroupId,queryChainId,queryGroupNumber,targetChainId
0,1XZ7,B,108,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,1XZ7.B,THR,A,108,B
1,1YDZ,B,108,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,1YDZ.B,THR,A,108,B
2,1Y09,D,108,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,1Y09.D,THR,C,108,D
3,1XYE,B,108,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,1XYE.B,THR,A,108,B
4,1XY0,B,108,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,1XY0.B,THR,A,108,B


In [9]:
chains = mt.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['1DXT.B', '1QXD.D', '1VWT.D', '1XXT.D', '1XY0.B', '1XYE.B', '1XZ2.D', '1XZ4.B', '1XZ5.B', '1XZ7.B', '1XZU.D', '1XZV.B', '1Y09.D', '1Y0A.B', '1Y0C.B', '1Y0D.B', '1Y8W.B', '1YDZ.B', '1YH9.B', '1YHE.D', '1YHR.B', '1YZI.B', '2W6V.D', '4XS0.B', '6FQF.D']


In [10]:
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

        # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
 
        # highlight chain of interest in blue
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
    
        # don't display water molecules
        viewer.setStyle({'resn': ['HOH','DOD']}, {})
        
        rows = group.shape[0]
        for j in range(0, rows):
            res_num = str(group.iloc[j]['pdbPosition'])
            mod_res = {'resi': res_num, 'chain': chain_id} 
            col = 'red'
            c_col = col + 'Carbon'
            viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
            viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})          
            if show_labels:
                label = group.iloc[j]['variationId']
                viewer.addLabel(label, {'fontSize':7,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})
        
        viewer.zoomTo({'chain': chain_id})
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        #print header
        print("PDB Id: " + pdb_id + " chain Id: " + chain_id)
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

In [11]:
view_grouped_mutations(chains);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

## View one mutation at a time
Use the slider to view each mutation. Interacting residues within the `distance_cutoff` are rendered as orange sticks.

In [12]:
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

        # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
        
        # highlight chain of interest in blue
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'opacity':0.7}})
        
        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # interacting residue style
        res_num = str(df.iloc[i]['pdbPosition'])
        label = df.iloc[i]['variationId']     
        mod_res = {'resi': res_num, 'chain': chain_id} 
        col = 'red'
        c_col = col + 'Carbon'
        viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
        viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.8}})   
        
        # don't display water molecules (except below for interactions)
        viewer.setStyle({'resn': ['HOH','DOD']}, {})
        
        #if show_labels:
        #    viewer.addLabel(label, {'fontSize':12,'fontColor': 'black','backgroundColor':'ivory'}, mod_res)
            
        # select residues in interacting chains by distance from mutation site (same chain is excluded)
        surroundings = {'not': {'chain': chain_id}, 'byres': True, 'within': {'distance': distance_cutoff, 'sel': mod_res}}
        
        # set style for interacting residues
        viewer.setStyle(surroundings,{'cartoon':{'color':'orange', 'width': 0.6}})
        viewer.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
        
        if show_labels:
            viewer.addResLabels(surroundings, {'fontSize':10})
            viewer.addLabel(label, {'fontSize':12,'fontColor': 'black','backgroundColor':'ivory'}, mod_res)
            
        viewer.zoomTo(surroundings)
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
                
        #print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "mutation:", label)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

In [13]:
view_single_mutation(mt, distance_cutoff);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [14]:
# Shutdown Spark
spark.stop()

## Now run the next step
Map mutations occuring at protein-ligand binding sites: [4-MapToLigandInteractions.ipynb](4-MapToLigandInteractions.ipynb)